# Performance by product --> SoM Dashboard

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Brands/data/'
CPG = 'sigma' 
nielsen_cat = False #True JUST FOR RB_HYHO, DANONE, UNILEVER SO FAR!!!
past = False
SOM_CPG = CPG.title()

## Import

In [3]:
import os
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
print([x for x in os.listdir(BASE_DIR) if x.endswith(f'{CPG}.csv')])

['products_sigma.csv', 'nielsen_sigma.csv', 'pasillos_sigma.csv', 'som_sigma.csv']


In [4]:
df = pd.read_csv(os.path.join(BASE_DIR,f'products_{CPG}.csv'),
                 sep='\t', encoding='UTF-16')
df.sample(2)

/Users/efraflores/Desktop/hub/cornershop/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (13,15,16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,sales,orders,avg ticket,users,units found,units replaced,units requested,found rate,fulfillment,frequency
6957,julio 2021,MX,Ciudad de México,City Market,Roast Beef & Pastrami,1630,1152086,Roast beef,{2624764000007},1 kg.,...,"4,348",16,272,13,6.608,0,7,100.0%,100.0%,1.23
29147,julio 2021,MX,Querétaro,Chedraui,Cheeses,28,323014,Queso panela,{7501314910205},400 g,...,"3,259",29,112,24,53,0,53,100.0%,100.0%,1.21


In [5]:
aux_ini = df.copy()
aux_ini['sales'] = aux_ini['sales'].astype(str).str.replace(',','').astype(float)
aux_ini = aux_ini.groupby('month  ')['sales'].sum().reset_index()
aux_ini

,month,sales
0,julio 2021,63950801.00


In [6]:
import re
nielsen = pd.read_csv(BASE_DIR + f'nielsen_{CPG}.csv',
                      sep='\t',encoding='utf-16'
                     ).drop_duplicates(subset=['UPC','PRODUCT ID'])
display(nielsen['UPC'].head())
nielsen['UPC'] = nielsen['UPC'].apply(lambda x: re.sub('[^\d]','',str(x)))
nielsen['UPC'] = nielsen['UPC'].astype(str).str.strip().str.split().str[0].fillna(0).astype(int)
display(nielsen['UPC'].head())
display(nielsen['UPC'].head())
nielsen.sample(2)

0    7622210265890
1     632593151250
2      46100010914
3    7501077513941
4    7503005898571
Name: UPC, dtype: object

0    7622210265890
1     632593151250
2      46100010914
3    7501077513941
4    7503005898571
Name: UPC, dtype: int64

0    7622210265890
1     632593151250
2      46100010914
3    7501077513941
4    7503005898571
Name: UPC, dtype: int64

,UPC,FORMATO,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,PRODUCT ID,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
2643,0,NaN,without CPG,NaN,NaN,Obertal,A granel,Pechuga cocida,1900831,Turkey Hams,NaN,NaN,NaN
851,37600314053,NaN,without CPG,NaN,NaN,Hormel,340 g,Tocino ahumado cherrywood,1954580,Sausages & Bacon,NaN,NaN,NaN


## Functions

In [7]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [8]:
def limpio(data,month_col,to_drop):
    df = data.copy()
    df[['mes','anio']] = df[month_col].str.split(expand=True)
    meses = dict(zip(['enero','febrero','marzo','abril','mayo','junio','julio',
                      'agosto','septiembre','octubre','noviembre','diciembre'],range(1,13)))
    df['mes'] = df['mes'].replace(meses)
    df.drop(columns=to_drop,inplace=True)
    df.columns = [str(x).replace('.','').replace(' ','_').lower() for x in df.columns]
    for col in ['brand_filter','cpg_filter']:
        df[f'{col[:-7]}_id'] = df[col].apply(lambda x: ','.join(re.findall('\s\((\d+)\)',str(x))))
        df[f'{col[:-7]}_name'] = df[col].apply(lambda x: re.sub('\s\(\d+\)','',str(x)))
    return df

## Transform

### Barcodes

In [9]:
df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')
df['barcodes'] = df['barcodes'].str.split(',',n=1).fillna('.').apply(lambda x:x[0])
df['barcodes'].sample(4)

/var/folders/j4/1p33_bh96yn8pdz3b_5t80hm0000gn/T/ipykernel_1462/2845833164.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')


27546    7503001475486
2718     7501055912711
35825    7501055914500
1141     7503013102080
Name: barcodes, dtype: object

### Merge

In [10]:
df = df.astype(str).merge(nielsen.astype(str).rename(columns={'UPC':'barcodes',
                                                              'PRODUCT ID':'product id'}),
                          on=['barcodes'] if nielsen_cat else ['barcodes','product id'],
                          how='left' if nielsen_cat else 'left')
if nielsen_cat:
    df = df.rename(columns={'product id_x':'product_id'}).drop(columns='product id_y')

df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
33143,julio 2021,MX,Querétaro,Soriana,Cheeses,28,343148,Queso manchego reducido en grasa,7501092114215,400 g,...,Lyncott,nan,nan,Lyncott,400 g,Queso manchego reducido en grasa,Cheeses,nan,nan,nan
7180,julio 2021,MX,Ciudad de México,City Market,Sausages & Bacon,1011,357696,Salchicha viena de res beef frankfuters,7501134500495,340 g,...,without CPG,nan,nan,Kurson Kosher,340 g,Salchicha viena de res beef frankfuters,Sausages & Bacon,nan,nan,nan


### CPG

In [11]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
julio 2021,63950801.00,63950801.00,0.00,0.00


In [12]:
prod_cpg = pd.read_csv(os.path.join(BASE_DIR,'pbbc.csv'),
                       #sep='\t',encoding='utf-16'
                      )
prod_cpg.sample(4)

,n,product_id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name
32699,1,221731,7501071121678,36,Gaseosas,6027,Squirt,191,Grupo Peñafiel
6067,1,112092,7591083017830,967,Higiene oral,556,Colgate,59,Colgate Palmolive
41549,1,319665,7501032905767,990,Ambientadores y velas,338,Glade,10,SC Johnson
65643,1,594661,70330194816,277,Escritura,448,BiC,292,BIC


In [13]:
prod_cpg.drop_duplicates(('barcodes' if nielsen_cat else 'product_id'),inplace=True)
prod_cpg = prod_cpg.astype(str).rename(columns={'product_id':'product id'})
prod_cpg['cpg filter'] = prod_cpg['cpg_name']+" ("+prod_cpg['cpg_id'].astype(str)+")"
prod_cpg.sample(4)

,n,product id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name,cpg filter
4108,1,101656,7501791632713,75,Especias y condimentos,37,Great Value,295,Walmex,Walmex (295)
14240,1,145454,7501046110003,964,Lociones y jabones,5221,Maja,164,Grisi,Grisi (164)
50636,1,342685,7501036622400,1583,Papel higiénico,5766,Regio,48,Essity,Essity (48)
58097,1,375709,8410234531065,83,Carnes y mariscos en conserva,1319,Vigilante,359,Mercantil Mx,Mercantil Mx (359)


In [14]:
df = df.merge(prod_cpg[[('barcodes' if nielsen_cat else 'product id'),'cpg filter']],
              on=('barcodes' if nielsen_cat else 'product id'),how='left')

In [15]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
julio 2021,63950801.00,63950801.00,0.00,0.00


### Limpio

In [16]:
df = limpio(df,'month  ',
            ['month  ','country ','avg ticket','found rate','fulfillment','frequency'])
df.sample(2)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id,cpg_name
2601,Cancún,Soriana,Yogurt,24,335910,Queso petit suisse con fresa,7501032335052,Pack 4 x 84 g,nan,Danonino (4348),...,nan,nan,nan,Danone (75),7,2021,4348,Danonino,75,Danone
24244,Monterrey,HEB,Special Cheeses,1617,339176,Queso ricotta con latte,31142001839,453 g,nan,Belgioioso (3931),...,nan,nan,nan,NaN,7,2021,3931,Belgioioso,,nan


In [17]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
jul,63950801.00,63950801.00,0.00,0.00


In [18]:
aux = df[df['formato'].astype(str)=='nan'].copy()
df = df[~(df['formato'].astype(str)=='nan')].copy()
null_fill = {'proveedor_general':'cpg_name',
             'proveedor_abierto':'cpg_name',
             'marca_detalle':'brand_name',
             'marca_generica':'brand_name',
             'gramaje':'package',
             'descripcion_nielsen':'product_name',
             'segmento':'category_en'}
for x,y in null_fill.items():
    aux.loc[:,x] = aux.loc[:,y]
    
df = df.append(aux,ignore_index=True)

In [19]:
for col in df.columns:
    df[col] = df[col].astype(str).str.replace('nan','')
    
df = df.drop(columns='cpg_name').fillna('')
df.sample()

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id
9456,Ciudad de México,La Comer,Cheeses,28,141306,Queso de cabra con arandano rojo,7501227200936,250 grs,7501227200936,Carol (4130),...,Cheeses,,,,,7,2021,4130,Carol,


In [20]:
brand_cols = ['formato','proveedor_general','proveedor_abierto','marca_detalle','marca_generica','gramaje','descripcion_nielsen','segmento','subsegmento_1','subsegmento_2','subsegmento_3']
display(df[brand_cols].sample())

for col in brand_cols:
    df[col] = df[col].str.strip().str.title()

display(df[brand_cols].sample())

,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
12868,,,,La Primavera de Chihuahua,La Primavera de Chihuahua,1 kg,Queso menonita tipo asadero,Cheeses,,,


,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
2609,,Sigma Total,Sigma Total,Yoplait,Yoplait,145 G,Yogurt Griego Natural Sin Azúcar,Yogurt,,,


In [21]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux_fin

,mes,sales
0,jul,63950801.00


## Export

### Same product_id as past

In [22]:
print(len(df))
if past:
    som = pd.ExcelFile(os.path.join('/'.join(BASE_DIR.split('/')[:-2]),
                                    f'SoM_{SOM_CPG}.xlsx')).parse(sheet_name=0)
    df = df.merge(som.drop_duplicates('product_id')[['product_id']].reset_index(drop=True).astype(str))
    
print(len(df))

37296
37296


In [23]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))

In [24]:
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1

### Ordered columns

In [25]:
export = df[['city_name', 'store_name', 'category_en', 'category_id', 'product_id',
             'product_name', 'barcodes', 'package', 'product_sku', 'brand_name',
             'sales', 'orders', 'users', 'units_found', 'units_replaced',
             'units_requested', 'formato', 'proveedor_general', 'proveedor_abierto',
             'marca_detalle', 'marca_generica', 'gramaje', 'descripcion_nielsen',
             'segmento', 'subsegmento_1', 'subsegmento_2', 'subsegmento_3',
             'mes', 'anio']].copy()
export.sort_values(['anio','mes'],inplace=True)
export.sample(4)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_name,...,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3,mes,anio
29393,Querétaro,Chedraui,Chicken,1009,343942,Pechugas de pollo rellenas de queso y jamón,7501100662356,490 g,,Del Día,...,Del Día,Del Día,490 G,Pechugas De Pollo Rellenas De Queso Y Jamón,Chicken,,,,7,2021
0,Aguascalientes,Chedraui,Cheeses,28,2356286,Queso crema original,7622210265890,200 g,442495,Philadelphia,...,Philadelphia,Philadelphia,200 G,Queso Crema Original,Cheeses,,,,7,2021
17655,Guadalajara,Soriana,Turkey Hams,1620,312959,Pechuga de pavo,7501057710773,250 g,,Zwan Bienestar,...,Zwan Bienestar,Zwan Bienestar,250 G,Pechuga De Pavo,Turkey Hams,,,,7,2021
3291,Chihuahua,Alsuper Plus,Cheeses,28,1232763,Queso Asadero Los Cedros Barra 1.1Kg,2800661000009,,415218,Los Cedros,...,Los Cedros,Los Cedros,,Queso Asadero Los Cedros Barra 1.1Kg,Cheeses,,,,7,2021


In [26]:
export.to_csv(os.path.join(BASE_DIR,f'som_{CPG}.csv'),index=False,sep='\t',encoding='utf-16')

## Fin

In [27]:
aux

,sales,sales_fin,dif,dif_p
month,,,,
jul,63950801.00,63950801.00,0.00,0.00


In [28]:
time_exp(time.time() - start)
tono()

0 minutos con 5.69 segundos
